# 期权与永续合约组合策略

**加密货币衍生品高阶交易指南**

---

## 目录

### 第一部分：期权基础
1. 期权合约简介
2. 看涨/看跌期权
3. Black-Scholes 定价模型
4. 希腊字母
5. 加密期权市场

### 第二部分：期权策略
6. 基础策略
7. 价差策略
8. 跨式策略
9. 蝶式/铁鹰策略

### 第三部分：期权+永续组合
10. Delta 对冲
11. 永续+期权组合
12. 资金费率+期权保护
13. Covered Call / Protective Put

### 第四部分：实战应用
14. 期权数据获取
15. 波动率分析
16. 希腊字母计算
17. 组合回测
18. 风险管理

---

# 第一部分：期权基础

---

## 1. 期权合约简介

### 1.1 什么是期权？

**期权（Option）** 是一种衍生品合约，赋予持有者在特定日期或之前以特定价格买入或卖出标的资产的权利（而非义务）。

### 1.2 期权的基本要素

| 要素 | 描述 |
|------|------|
| **标的资产** | 期权对应的资产（如 BTC） |
| **行权价** | 约定的买卖价格 |
| **到期日** | 期权失效的日期 |
| **期权费** | 购买期权支付的费用 |
| **期权类型** | 看涨（Call）或看跌（Put） |

### 1.3 期权 vs 永续合约

```
┌─────────────────────────────────────────────────────────────────┐
│                    期权 vs 永续合约对比                         │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  期权 (Options)                  永续合约 (Perpetual)          │
│  ├── 有到期日                    ├── 无到期日                  │
│  ├── 付期权费，风险有限          ├── 付资金费率，风险无限      │
│  ├── 非线性收益                  ├── 线性收益                  │
│  ├── 可用于对冲波动率            ├── 主要用于方向性交易        │
│  └── 流动性相对较差              └── 流动性好                  │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

## 2. 看涨/看跌期权

### 2.1 看涨期权（Call Option）

**买入看涨期权**：预期价格上涨
- 最大亏损 = 期权费
- 最大收益 = 无限

**卖出看涨期权**：预期价格不会大涨
- 最大收益 = 期权费
- 最大亏损 = 无限

### 2.2 看跌期权（Put Option）

**买入看跌期权**：预期价格下跌
- 最大亏损 = 期权费
- 最大收益 = 行权价 - 期权费

**卖出看跌期权**：预期价格不会大跌
- 最大收益 = 期权费
- 最大亏损 = 行权价 - 期权费

### 2.3 期权收益图

```
买入看涨期权 (Long Call)           买入看跌期权 (Long Put)

    收益                              收益
    │      ╱                          │  ╲
    │     ╱                           │   ╲
────┼────●────────── 价格         ────┼────●────────── 价格
    │   行权价                        │   行权价
    │                                 │

卖出看涨期权 (Short Call)          卖出看跌期权 (Short Put)

    收益                              收益
    │                                 │
────┼────●────────── 价格         ────┼────●────────── 价格
    │     ╲                           │   ╱
    │      ╲                          │  ╱
```

## 3. Black-Scholes 定价模型

### 3.1 模型公式

**看涨期权价格**：
$$C = S_0 N(d_1) - K e^{-rT} N(d_2)$$

**看跌期权价格**：
$$P = K e^{-rT} N(-d_2) - S_0 N(-d_1)$$

其中：
$$d_1 = \frac{\ln(S_0/K) + (r + \sigma^2/2)T}{\sigma\sqrt{T}}$$
$$d_2 = d_1 - \sigma\sqrt{T}$$

**参数说明**：
- $S_0$: 标的资产当前价格
- $K$: 行权价格
- $T$: 到期时间（年）
- $r$: 无风险利率
- $\sigma$: 波动率
- $N(x)$: 标准正态分布累积函数

In [ ]:
# 3.2 Black-Scholes 实现

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.stats import norm
from scipy.optimize import brentq
import warnings

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-whitegrid')

def black_scholes(S, K, T, r, sigma, option_type='call'):
    """
    Black-Scholes 期权定价
    
    参数:
        S: 标的价格
        K: 行权价
        T: 到期时间（年）
        r: 无风险利率
        sigma: 波动率
        option_type: 'call' 或 'put'
    """
    if T <= 0:
        # 到期时的价值
        if option_type == 'call':
            return max(S - K, 0)
        else:
            return max(K - S, 0)
    
    d1 = (np.log(S / K) + (r + sigma**2 / 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    if option_type == 'call':
        price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    else:
        price = K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    
    return price

# 示例计算
S = 50000  # BTC 价格
K = 52000  # 行权价
T = 30/365  # 30天
r = 0.05  # 5% 年化
sigma = 0.6  # 60% 波动率

call_price = black_scholes(S, K, T, r, sigma, 'call')
put_price = black_scholes(S, K, T, r, sigma, 'put')

print('📊 Black-Scholes 期权定价')
print('=' * 40)
print(f'标的价格: ${S:,}')
print(f'行权价: ${K:,}')
print(f'到期时间: {T*365:.0f} 天')
print(f'波动率: {sigma*100:.0f}%')
print('-' * 40)
print(f'看涨期权价格: ${call_price:,.2f}')
print(f'看跌期权价格: ${put_price:,.2f}')

## 4. 希腊字母

In [ ]:
# 4.1 希腊字母计算

def calculate_greeks(S, K, T, r, sigma, option_type='call'):
    """
    计算期权的希腊字母
    """
    if T <= 0:
        return {'delta': 0, 'gamma': 0, 'theta': 0, 'vega': 0, 'rho': 0}
    
    d1 = (np.log(S / K) + (r + sigma**2 / 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    # Delta: 价格敏感度
    if option_type == 'call':
        delta = norm.cdf(d1)
    else:
        delta = norm.cdf(d1) - 1
    
    # Gamma: Delta 的变化率
    gamma = norm.pdf(d1) / (S * sigma * np.sqrt(T))
    
    # Theta: 时间衰减（每天）
    theta_part1 = -S * norm.pdf(d1) * sigma / (2 * np.sqrt(T))
    if option_type == 'call':
        theta = (theta_part1 - r * K * np.exp(-r * T) * norm.cdf(d2)) / 365
    else:
        theta = (theta_part1 + r * K * np.exp(-r * T) * norm.cdf(-d2)) / 365
    
    # Vega: 波动率敏感度（1%变化）
    vega = S * norm.pdf(d1) * np.sqrt(T) / 100
    
    # Rho: 利率敏感度（1%变化）
    if option_type == 'call':
        rho = K * T * np.exp(-r * T) * norm.cdf(d2) / 100
    else:
        rho = -K * T * np.exp(-r * T) * norm.cdf(-d2) / 100
    
    return {
        'delta': delta,
        'gamma': gamma,
        'theta': theta,
        'vega': vega,
        'rho': rho
    }

# 计算希腊字母
greeks_call = calculate_greeks(S, K, T, r, sigma, 'call')
greeks_put = calculate_greeks(S, K, T, r, sigma, 'put')

print('📊 希腊字母')
print('=' * 50)
print(f'{"指标":<10} {"看涨期权":<15} {"看跌期权":<15}')
print('-' * 50)
print(f'{"Delta":<10} {greeks_call["delta"]:>12.4f}   {greeks_put["delta"]:>12.4f}')
print(f'{"Gamma":<10} {greeks_call["gamma"]:>12.6f}   {greeks_put["gamma"]:>12.6f}')
print(f'{"Theta":<10} {greeks_call["theta"]:>12.2f}   {greeks_put["theta"]:>12.2f}')
print(f'{"Vega":<10} {greeks_call["vega"]:>12.2f}   {greeks_put["vega"]:>12.2f}')
print(f'{"Rho":<10} {greeks_call["rho"]:>12.2f}   {greeks_put["rho"]:>12.2f}')
print('=' * 50)

### 4.2 希腊字母解读

```
┌─────────────────────────────────────────────────────────────────┐
│                        希腊字母解读                             │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  Delta (Δ)                                                     │
│  ├── 含义：标的价格变化$1，期权价格变化多少                    │
│  ├── 范围：Call [0,1], Put [-1,0]                              │
│  └── 应用：对冲比率                                            │
│                                                                 │
│  Gamma (Γ)                                                     │
│  ├── 含义：Delta 的变化率                                      │
│  ├── 特点：平值期权 Gamma 最大                                 │
│  └── 应用：衡量 Delta 对冲的频率需求                           │
│                                                                 │
│  Theta (Θ)                                                     │
│  ├── 含义：时间流逝导致的价值损耗（每天）                      │
│  ├── 特点：通常为负（期权买方损失）                            │
│  └── 应用：时间价值管理                                        │
│                                                                 │
│  Vega (ν)                                                      │
│  ├── 含义：波动率变化1%，期权价格变化多少                      │
│  ├── 特点：平值期权 Vega 最大                                  │
│  └── 应用：波动率交易                                          │
│                                                                 │
│  Rho (ρ)                                                       │
│  ├── 含义：利率变化1%，期权价格变化多少                        │
│  └── 应用：加密市场影响较小                                    │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

## 5. 加密期权市场

### 5.1 主要交易所

| 交易所 | 特点 | 合约类型 |
|--------|------|----------|
| **Deribit** | 市场份额最大 | 欧式期权 |
| **OKX** | 流动性较好 | 欧式期权 |
| **Binance** | 新进入者 | 欧式期权 |
| **CME** | 机构为主 | 美式期权 |

### 5.2 加密期权特点

- 波动率高（50%-100%）
- 24/7 交易
- 欧式期权为主
- 结算货币：BTC/USDT

---

# 第二部分：期权策略

---

## 6. 基础策略

In [ ]:
# 6.1 期权收益计算器

class OptionPayoff:
    """
    期权收益计算器
    """
    @staticmethod
    def long_call(S, K, premium):
        """买入看涨期权"""
        return np.maximum(S - K, 0) - premium
    
    @staticmethod
    def short_call(S, K, premium):
        """卖出看涨期权"""
        return -np.maximum(S - K, 0) + premium
    
    @staticmethod
    def long_put(S, K, premium):
        """买入看跌期权"""
        return np.maximum(K - S, 0) - premium
    
    @staticmethod
    def short_put(S, K, premium):
        """卖出看跌期权"""
        return -np.maximum(K - S, 0) + premium

def plot_payoff(strategies: list, price_range: tuple, title: str = '期权收益图'):
    """
    绘制期权收益图
    """
    prices = np.linspace(price_range[0], price_range[1], 100)
    
    fig = go.Figure()
    
    total_payoff = np.zeros_like(prices)
    
    for strategy in strategies:
        payoff = strategy['payoff_func'](prices, strategy['K'], strategy['premium'])
        total_payoff += payoff * strategy.get('quantity', 1)
        
        fig.add_trace(go.Scatter(
            x=prices,
            y=payoff,
            mode='lines',
            name=strategy['name'],
            line=dict(dash='dash')
        ))
    
    fig.add_trace(go.Scatter(
        x=prices,
        y=total_payoff,
        mode='lines',
        name='组合收益',
        line=dict(color='black', width=3)
    ))
    
    fig.add_hline(y=0, line_dash='dot', line_color='gray')
    
    fig.update_layout(
        title=title,
        xaxis_title='标的价格',
        yaxis_title='收益/亏损',
        height=400
    )
    
    return fig

# 示例：买入看涨期权
strategies = [
    {'name': '买入 Call', 'payoff_func': OptionPayoff.long_call, 'K': 52000, 'premium': 2000}
]

fig = plot_payoff(strategies, (40000, 65000), '买入看涨期权')
fig.show()

## 7. 价差策略

In [ ]:
# 7.1 牛市价差（Bull Call Spread）

# 买入低行权价 Call + 卖出高行权价 Call
strategies = [
    {'name': '买入 Call K=50000', 'payoff_func': OptionPayoff.long_call, 'K': 50000, 'premium': 3000},
    {'name': '卖出 Call K=55000', 'payoff_func': OptionPayoff.short_call, 'K': 55000, 'premium': 1500}
]

fig = plot_payoff(strategies, (40000, 65000), '牛市看涨价差（Bull Call Spread）')
fig.show()

print('📊 牛市价差策略')
print('=' * 40)
print('构建：买入低行权价 Call + 卖出高行权价 Call')
print(f'净成本: ${3000-1500:,}')
print(f'最大收益: ${55000-50000-1500:,}')
print(f'最大亏损: ${1500:,}')
print('适用场景: 温和看涨')

In [ ]:
# 7.2 熊市价差（Bear Put Spread）

# 买入高行权价 Put + 卖出低行权价 Put
strategies = [
    {'name': '买入 Put K=50000', 'payoff_func': OptionPayoff.long_put, 'K': 50000, 'premium': 2500},
    {'name': '卖出 Put K=45000', 'payoff_func': OptionPayoff.short_put, 'K': 45000, 'premium': 1000}
]

fig = plot_payoff(strategies, (35000, 60000), '熊市看跌价差（Bear Put Spread）')
fig.show()

print('📊 熊市价差策略')
print('=' * 40)
print('构建：买入高行权价 Put + 卖出低行权价 Put')
print(f'净成本: ${2500-1000:,}')
print(f'最大收益: ${50000-45000-1500:,}')
print(f'最大亏损: ${1500:,}')
print('适用场景: 温和看跌')

## 8. 跨式策略

In [ ]:
# 8.1 跨式组合（Straddle）

K = 50000
call_premium = 2500
put_premium = 2500

strategies = [
    {'name': '买入 Call', 'payoff_func': OptionPayoff.long_call, 'K': K, 'premium': call_premium},
    {'name': '买入 Put', 'payoff_func': OptionPayoff.long_put, 'K': K, 'premium': put_premium}
]

fig = plot_payoff(strategies, (35000, 65000), '跨式组合（Long Straddle）')
fig.show()

total_premium = call_premium + put_premium
upper_breakeven = K + total_premium
lower_breakeven = K - total_premium

print('📊 跨式组合策略')
print('=' * 40)
print('构建：同时买入相同行权价的 Call 和 Put')
print(f'总成本: ${total_premium:,}')
print(f'上方盈亏平衡: ${upper_breakeven:,}')
print(f'下方盈亏平衡: ${lower_breakeven:,}')
print(f'最大亏损: ${total_premium:,}')
print('适用场景: 预期大幅波动，方向不确定')

In [ ]:
# 8.2 宽跨式组合（Strangle）

strategies = [
    {'name': '买入 Call K=52000', 'payoff_func': OptionPayoff.long_call, 'K': 52000, 'premium': 1500},
    {'name': '买入 Put K=48000', 'payoff_func': OptionPayoff.long_put, 'K': 48000, 'premium': 1500}
]

fig = plot_payoff(strategies, (35000, 65000), '宽跨式组合（Long Strangle）')
fig.show()

print('📊 宽跨式组合策略')
print('=' * 40)
print('构建：买入虚值 Call + 买入虚值 Put')
print('成本比 Straddle 低，但需要更大波动才能盈利')
print('适用场景: 预期大幅波动，成本控制')

## 9. 蝶式/铁鹰策略

In [ ]:
# 9.1 蝶式组合（Butterfly）

strategies = [
    {'name': '买入 Call K=48000', 'payoff_func': OptionPayoff.long_call, 'K': 48000, 'premium': 3500, 'quantity': 1},
    {'name': '卖出 Call K=50000', 'payoff_func': OptionPayoff.short_call, 'K': 50000, 'premium': 2500, 'quantity': 2},
    {'name': '买入 Call K=52000', 'payoff_func': OptionPayoff.long_call, 'K': 52000, 'premium': 1500, 'quantity': 1}
]

fig = plot_payoff(strategies, (42000, 58000), '蝶式组合（Butterfly Spread）')
fig.show()

print('📊 蝶式组合策略')
print('=' * 40)
print('构建：买1低K Call + 卖2中K Call + 买1高K Call')
print('最大收益在中间行权价附近')
print('适用场景: 预期价格在窄幅区间波动')

In [ ]:
# 9.2 铁鹰组合（Iron Condor）

strategies = [
    {'name': '买入 Put K=45000', 'payoff_func': OptionPayoff.long_put, 'K': 45000, 'premium': 500},
    {'name': '卖出 Put K=48000', 'payoff_func': OptionPayoff.short_put, 'K': 48000, 'premium': 1000},
    {'name': '卖出 Call K=52000', 'payoff_func': OptionPayoff.short_call, 'K': 52000, 'premium': 1000},
    {'name': '买入 Call K=55000', 'payoff_func': OptionPayoff.long_call, 'K': 55000, 'premium': 500}
]

fig = plot_payoff(strategies, (40000, 60000), '铁鹰组合（Iron Condor）')
fig.show()

print('📊 铁鹰组合策略')
print('=' * 40)
print('构建：牛市Put价差 + 熊市Call价差')
print('收取权利金，价格在区间内即盈利')
print('适用场景: 预期价格在宽幅区间内波动')

---

# 第三部分：期权+永续组合

---

## 10. Delta 对冲

In [ ]:
# 10.1 Delta 对冲计算器

class DeltaHedger:
    """
    Delta 对冲计算器
    """
    def __init__(self, S, K, T, r, sigma, option_type='call', position_size=1):
        self.S = S
        self.K = K
        self.T = T
        self.r = r
        self.sigma = sigma
        self.option_type = option_type
        self.position_size = position_size
    
    def calculate_hedge_ratio(self):
        """
        计算对冲比率（需要的永续合约数量）
        """
        greeks = calculate_greeks(self.S, self.K, self.T, self.r, self.sigma, self.option_type)
        delta = greeks['delta']
        
        # 如果持有 Long Option，需要做空对冲
        hedge_ratio = -delta * self.position_size
        
        return {
            'delta': delta,
            'hedge_ratio': hedge_ratio,
            'hedge_direction': '做空' if hedge_ratio > 0 else '做多',
            'hedge_size': abs(hedge_ratio)
        }
    
    def simulate_hedged_pnl(self, price_changes: np.ndarray) -> dict:
        """
        模拟对冲后的收益
        """
        hedge_info = self.calculate_hedge_ratio()
        hedge_size = hedge_info['hedge_size']
        
        option_pnls = []
        hedge_pnls = []
        net_pnls = []
        
        current_S = self.S
        
        for dS in price_changes:
            new_S = current_S + dS
            
            # 期权 P&L
            old_price = black_scholes(current_S, self.K, self.T, self.r, self.sigma, self.option_type)
            new_price = black_scholes(new_S, self.K, self.T, self.r, self.sigma, self.option_type)
            option_pnl = (new_price - old_price) * self.position_size
            
            # 对冲 P&L
            if hedge_info['hedge_direction'] == '做空':
                hedge_pnl = -dS * hedge_size
            else:
                hedge_pnl = dS * hedge_size
            
            option_pnls.append(option_pnl)
            hedge_pnls.append(hedge_pnl)
            net_pnls.append(option_pnl + hedge_pnl)
            
            current_S = new_S
        
        return {
            'option_pnl': np.array(option_pnls),
            'hedge_pnl': np.array(hedge_pnls),
            'net_pnl': np.array(net_pnls)
        }

# 示例
hedger = DeltaHedger(S=50000, K=52000, T=30/365, r=0.05, sigma=0.6, option_type='call', position_size=1)
hedge_info = hedger.calculate_hedge_ratio()

print('📊 Delta 对冲')
print('=' * 40)
print(f'期权 Delta: {hedge_info["delta"]:.4f}')
print(f'对冲方向: {hedge_info["hedge_direction"]}')
print(f'对冲数量: {hedge_info["hedge_size"]:.4f} BTC')
print('\n💡 持有1个 Long Call，需要做空 {:.4f} BTC 来对冲'.format(hedge_info['hedge_size']))

## 11. 永续+期权组合

In [ ]:
# 11.1 资金费率套利 + 期权保护

class FundingArbitrageWithOptions:
    """
    资金费率套利 + 期权保护策略
    
    策略：
    - 现货做多 + 永续做空（收取正资金费率）
    - 买入看跌期权（防止暴跌）
    """
    def __init__(self, spot_size: float, spot_price: float, put_strike: float, put_premium: float):
        self.spot_size = spot_size
        self.spot_price = spot_price
        self.put_strike = put_strike
        self.put_premium = put_premium
        self.total_investment = spot_size * spot_price + put_premium
    
    def calculate_payoff(self, final_price: float, funding_income: float) -> dict:
        """
        计算最终收益
        """
        # 现货收益
        spot_pnl = (final_price - self.spot_price) * self.spot_size
        
        # 永续空头收益（价格变动被对冲）
        futures_pnl = -(final_price - self.spot_price) * self.spot_size
        
        # 期权收益
        put_pnl = max(self.put_strike - final_price, 0) * self.spot_size - self.put_premium
        
        # 资金费率收入
        funding_pnl = funding_income
        
        total_pnl = spot_pnl + futures_pnl + put_pnl + funding_pnl
        
        return {
            'spot_pnl': spot_pnl,
            'futures_pnl': futures_pnl,
            'put_pnl': put_pnl,
            'funding_pnl': funding_pnl,
            'total_pnl': total_pnl,
            'return_pct': total_pnl / self.total_investment * 100
        }

# 示例
strategy = FundingArbitrageWithOptions(
    spot_size=1,
    spot_price=50000,
    put_strike=48000,
    put_premium=1000
)

print('📊 资金费率套利 + 期权保护')
print('=' * 50)
print('策略构建：')
print('  - 买入 1 BTC 现货')
print('  - 做空 1 BTC 永续合约')
print('  - 买入 1 个行权价 $48,000 的看跌期权')
print(f'\n总投资: ${strategy.total_investment:,}')
print('\n不同情景下的收益：')

for price in [40000, 45000, 50000, 55000, 60000]:
    result = strategy.calculate_payoff(price, funding_income=500)  # 假设收取 $500 资金费率
    print(f'  价格 ${price:,}: 总收益 ${result["total_pnl"]:,.0f} ({result["return_pct"]:.2f}%)')

## 12. 资金费率+期权保护

```
┌─────────────────────────────────────────────────────────────────┐
│               资金费率套利 + 期权保护策略                        │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│   基础策略：现货做多 + 永续做空                                 │
│   ├── 收益：正资金费率（约 10-30% 年化）                       │
│   └── 风险：基差变动、极端行情下的强平风险                     │
│                                                                 │
│   期权保护：买入虚值看跌期权                                    │
│   ├── 成本：期权费（约 1-3% 每月）                             │
│   └── 保护：限制下跌损失                                       │
│                                                                 │
│   组合效果：                                                    │
│   ├── 上涨：收取资金费率，期权作废                             │
│   ├── 横盘：收取资金费率，期权作废                             │
│   └── 暴跌：期权提供保护，限制损失                             │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

## 13. Covered Call / Protective Put

In [ ]:
# 13.1 Covered Call（持有现货 + 卖出看涨期权）

spot_price = 50000
call_strike = 55000
call_premium = 1500

prices = np.linspace(40000, 65000, 100)
spot_payoff = prices - spot_price
call_payoff = -np.maximum(prices - call_strike, 0) + call_premium
covered_call_payoff = spot_payoff + call_payoff

fig = go.Figure()
fig.add_trace(go.Scatter(x=prices, y=spot_payoff, name='现货', line=dict(dash='dash')))
fig.add_trace(go.Scatter(x=prices, y=call_payoff, name='卖出 Call', line=dict(dash='dash')))
fig.add_trace(go.Scatter(x=prices, y=covered_call_payoff, name='Covered Call', line=dict(width=3, color='black')))
fig.add_hline(y=0, line_dash='dot', line_color='gray')
fig.update_layout(title='Covered Call 策略', xaxis_title='价格', yaxis_title='收益', height=400)
fig.show()

print('📊 Covered Call 策略')
print('=' * 40)
print('构建：持有现货 + 卖出虚值看涨期权')
print(f'最大收益: ${call_strike - spot_price + call_premium:,}')
print('适用场景: 温和看涨，愿意在高位卖出')

In [ ]:
# 13.2 Protective Put（持有现货 + 买入看跌期权）

spot_price = 50000
put_strike = 48000
put_premium = 1500

prices = np.linspace(35000, 65000, 100)
spot_payoff = prices - spot_price
put_payoff = np.maximum(put_strike - prices, 0) - put_premium
protective_put_payoff = spot_payoff + put_payoff

fig = go.Figure()
fig.add_trace(go.Scatter(x=prices, y=spot_payoff, name='现货', line=dict(dash='dash')))
fig.add_trace(go.Scatter(x=prices, y=put_payoff, name='买入 Put', line=dict(dash='dash')))
fig.add_trace(go.Scatter(x=prices, y=protective_put_payoff, name='Protective Put', line=dict(width=3, color='black')))
fig.add_hline(y=0, line_dash='dot', line_color='gray')
fig.update_layout(title='Protective Put 策略', xaxis_title='价格', yaxis_title='收益', height=400)
fig.show()

print('📊 Protective Put 策略')
print('=' * 40)
print('构建：持有现货 + 买入虚值看跌期权')
print(f'最大亏损: ${spot_price - put_strike + put_premium:,}')
print('适用场景: 看涨但需要下跌保护')

---

# 第四部分：实战应用

---

## 14. 期权数据获取

In [ ]:
# 14.1 期权数据获取示例

import ccxt

# Deribit 期权数据（需要 API 支持）
print('📊 期权数据获取示例')
print('=' * 40)
print('''
# Deribit 期权数据获取
exchange = ccxt.deribit()
markets = exchange.fetch_markets()

# 筛选 BTC 期权
btc_options = [m for m in markets if 'BTC' in m['symbol'] and m['type'] == 'option']

# 获取期权报价
option_ticker = exchange.fetch_ticker('BTC-30JUN23-50000-C')

# 获取期权链
option_chain = exchange.fetch_option_chain('BTC')
''')

print('\n💡 提示: 需要配置交易所 API 密钥')

## 15. 波动率分析

In [ ]:
# 15.1 隐含波动率计算

def implied_volatility(market_price, S, K, T, r, option_type='call'):
    """
    计算隐含波动率
    """
    def objective(sigma):
        return black_scholes(S, K, T, r, sigma, option_type) - market_price
    
    try:
        iv = brentq(objective, 0.01, 5.0)
        return iv
    except:
        return None

# 示例
market_price = 2500  # 市场期权价格
S = 50000
K = 52000
T = 30/365
r = 0.05

iv = implied_volatility(market_price, S, K, T, r, 'call')

print('📊 隐含波动率计算')
print('=' * 40)
print(f'市场价格: ${market_price:,}')
print(f'隐含波动率: {iv*100:.2f}%' if iv else '无法计算')

In [ ]:
# 15.2 波动率微笑

def plot_volatility_smile(S, T, r, strikes, market_prices, option_type='call'):
    """
    绘制波动率微笑
    """
    ivs = []
    for K, price in zip(strikes, market_prices):
        iv = implied_volatility(price, S, K, T, r, option_type)
        ivs.append(iv * 100 if iv else None)
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=strikes,
        y=ivs,
        mode='lines+markers',
        name='隐含波动率'
    ))
    fig.add_vline(x=S, line_dash='dash', line_color='gray', annotation_text='ATM')
    fig.update_layout(
        title='波动率微笑',
        xaxis_title='行权价',
        yaxis_title='隐含波动率 (%)',
        height=400
    )
    return fig

# 模拟数据
strikes = [45000, 47500, 50000, 52500, 55000]
market_prices = [5500, 3500, 2000, 1200, 700]  # 模拟的市场价格

fig = plot_volatility_smile(50000, 30/365, 0.05, strikes, market_prices)
fig.show()

## 16. 希腊字母计算

In [ ]:
# 16.1 希腊字母可视化

def plot_greeks(S_range, K, T, r, sigma, option_type='call'):
    """
    绘制希腊字母随价格变化
    """
    deltas, gammas, thetas, vegas = [], [], [], []
    
    for S in S_range:
        g = calculate_greeks(S, K, T, r, sigma, option_type)
        deltas.append(g['delta'])
        gammas.append(g['gamma'])
        thetas.append(g['theta'])
        vegas.append(g['vega'])
    
    fig = make_subplots(rows=2, cols=2, subplot_titles=('Delta', 'Gamma', 'Theta', 'Vega'))
    
    fig.add_trace(go.Scatter(x=S_range, y=deltas, name='Delta'), row=1, col=1)
    fig.add_trace(go.Scatter(x=S_range, y=gammas, name='Gamma'), row=1, col=2)
    fig.add_trace(go.Scatter(x=S_range, y=thetas, name='Theta'), row=2, col=1)
    fig.add_trace(go.Scatter(x=S_range, y=vegas, name='Vega'), row=2, col=2)
    
    for i in range(1, 3):
        for j in range(1, 3):
            fig.add_vline(x=K, line_dash='dash', line_color='gray', row=i, col=j)
    
    fig.update_layout(height=600, title=f'{option_type.title()} 期权希腊字母 (K={K})', showlegend=False)
    return fig

S_range = np.linspace(40000, 60000, 50)
fig = plot_greeks(S_range, K=50000, T=30/365, r=0.05, sigma=0.6, option_type='call')
fig.show()

## 17. 组合回测

In [ ]:
# 17.1 期权策略回测框架

class OptionsBacktest:
    """
    期权策略回测框架
    """
    def __init__(self, initial_capital: float = 100000):
        self.initial_capital = initial_capital
        self.capital = initial_capital
        self.positions = []
        self.history = []
    
    def add_position(self, option_type: str, K: float, T: float, premium: float, quantity: int = 1):
        """
        添加期权头寸
        """
        self.positions.append({
            'type': option_type,
            'K': K,
            'T': T,
            'premium': premium,
            'quantity': quantity
        })
        self.capital -= premium * quantity
    
    def calculate_portfolio_value(self, S: float, r: float = 0.05, sigma: float = 0.6) -> float:
        """
        计算组合价值
        """
        total_value = self.capital
        
        for pos in self.positions:
            option_value = black_scholes(S, pos['K'], pos['T'], r, sigma, pos['type'])
            total_value += option_value * pos['quantity']
        
        return total_value
    
    def run_backtest(self, price_series: np.ndarray, r: float = 0.05, sigma: float = 0.6) -> dict:
        """
        运行回测
        """
        portfolio_values = []
        
        for S in price_series:
            value = self.calculate_portfolio_value(S, r, sigma)
            portfolio_values.append(value)
            
            # 更新剩余时间
            for pos in self.positions:
                pos['T'] = max(0, pos['T'] - 1/365)
        
        final_value = portfolio_values[-1]
        total_return = (final_value - self.initial_capital) / self.initial_capital
        
        return {
            'portfolio_values': portfolio_values,
            'final_value': final_value,
            'total_return': total_return * 100,
            'max_drawdown': self._calculate_max_drawdown(portfolio_values)
        }
    
    def _calculate_max_drawdown(self, values: list) -> float:
        peak = values[0]
        max_dd = 0
        for v in values:
            if v > peak:
                peak = v
            dd = (peak - v) / peak
            if dd > max_dd:
                max_dd = dd
        return max_dd * 100

# 示例回测
bt = OptionsBacktest(initial_capital=100000)
bt.add_position('call', K=52000, T=30/365, premium=2500, quantity=5)

# 模拟价格序列
np.random.seed(42)
prices = 50000 + np.cumsum(np.random.randn(30) * 1000)

result = bt.run_backtest(prices)

print('📊 期权策略回测结果')
print('=' * 40)
print(f'初始资金: ${bt.initial_capital:,}')
print(f'最终价值: ${result["final_value"]:,.2f}')
print(f'总收益: {result["total_return"]:.2f}%')
print(f'最大回撤: {result["max_drawdown"]:.2f}%')

## 18. 风险管理

### 18.1 期权组合风险管理框架

```
┌─────────────────────────────────────────────────────────────────┐
│                    期权风险管理框架                             │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  1. Delta 风险管理                                             │
│     ├── 监控组合 Delta                                         │
│     ├── 使用永续合约对冲                                       │
│     └── 设置 Delta 中性阈值                                    │
│                                                                 │
│  2. Gamma 风险管理                                             │
│     ├── 避免过大的 Gamma 敞口                                  │
│     └── 在高 Gamma 时更频繁对冲                                │
│                                                                 │
│  3. Theta 风险管理                                             │
│     ├── 监控时间价值衰减                                       │
│     └── 避免持有过多长期权头寸                                 │
│                                                                 │
│  4. Vega 风险管理                                              │
│     ├── 监控波动率敞口                                         │
│     └── 使用波动率对冲工具                                     │
│                                                                 │
│  5. 整体风险控制                                               │
│     ├── 单一头寸限制：不超过组合的 20%                         │
│     ├── 最大亏损限制：总资金的 5%                              │
│     └── 流动性管理：保持 30% 现金                              │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

In [ ]:
# 18.2 组合风险监控

class PortfolioRiskMonitor:
    """
    组合风险监控器
    """
    def __init__(self, positions: list, S: float, r: float = 0.05, sigma: float = 0.6):
        self.positions = positions
        self.S = S
        self.r = r
        self.sigma = sigma
    
    def calculate_portfolio_greeks(self) -> dict:
        """
        计算组合希腊字母
        """
        total_delta = 0
        total_gamma = 0
        total_theta = 0
        total_vega = 0
        
        for pos in self.positions:
            greeks = calculate_greeks(self.S, pos['K'], pos['T'], self.r, self.sigma, pos['type'])
            quantity = pos['quantity']
            
            total_delta += greeks['delta'] * quantity
            total_gamma += greeks['gamma'] * quantity
            total_theta += greeks['theta'] * quantity
            total_vega += greeks['vega'] * quantity
        
        return {
            'delta': total_delta,
            'gamma': total_gamma,
            'theta': total_theta,
            'vega': total_vega
        }
    
    def check_risk_limits(self, max_delta: float = 1.0, max_theta: float = -500) -> dict:
        """
        检查风险限制
        """
        greeks = self.calculate_portfolio_greeks()
        
        alerts = []
        if abs(greeks['delta']) > max_delta:
            alerts.append(f"Delta 超限: {greeks['delta']:.4f}")
        if greeks['theta'] < max_theta:
            alerts.append(f"Theta 过大: {greeks['theta']:.2f}")
        
        return {
            'greeks': greeks,
            'alerts': alerts,
            'is_safe': len(alerts) == 0
        }

# 示例
positions = [
    {'type': 'call', 'K': 52000, 'T': 30/365, 'quantity': 5},
    {'type': 'put', 'K': 48000, 'T': 30/365, 'quantity': -3}
]

monitor = PortfolioRiskMonitor(positions, S=50000)
risk_report = monitor.check_risk_limits()

print('📊 组合风险报告')
print('=' * 40)
print(f"组合 Delta: {risk_report['greeks']['delta']:.4f}")
print(f"组合 Gamma: {risk_report['greeks']['gamma']:.6f}")
print(f"组合 Theta: {risk_report['greeks']['theta']:.2f}")
print(f"组合 Vega: {risk_report['greeks']['vega']:.2f}")
print(f"\n风险状态: {'✅ 安全' if risk_report['is_safe'] else '⚠️ 警告'}")
for alert in risk_report['alerts']:
    print(f"  - {alert}")

---

## 总结

### 核心要点

1. **期权基础**：
   - 期权提供非线性收益
   - Black-Scholes 模型是定价基础
   - 希腊字母量化各类风险

2. **常用策略**：
   - 价差策略：控制成本和风险
   - 跨式策略：交易波动率
   - 铁鹰策略：收取权利金

3. **期权+永续组合**：
   - Delta 对冲：消除方向风险
   - 保护性策略：限制下行风险
   - 增强收益：Covered Call

4. **风险管理**：
   - 监控组合希腊字母
   - 设置风险限制
   - 定期再平衡

### 实战建议

- 从简单策略开始
- 重视流动性
- 控制仓位规模
- 持续监控风险

### 学习完成！

恭喜你完成了资金费率与基差系统研究的全部进阶内容！